In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import json
import pprint as pp
import time

path_resource = r'./logo-resource/'
if not os.path.exists(path_resource):
    os.mkdir(path_resource)
    
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'} 

url_start = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp?tid=5f1da434-ad44-4955-97d3-1ac4433603f1(203.204.81.115)'
url_each = 'https://twtmsearch.tipo.gov.tw/OS0/OS0401_q.jsp'

for years in range(109,110):
    data_fore = {'state': 'queryResultJSONObject',
    'searchConditions': '{"datePubStart":"%s/01/01","datePubEnd":"%s/06/31","dptKind":"OPP","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'%(years,years)
           }
    data_back = {'state': 'queryResultJSONObject',
    'searchConditions': '{"datePubStart":"%s/07/01","datePubEnd":"%s/12/31","dptKind":"OPP","userType":"=====請選擇=====","boolean1":"=====請選擇=====","boolean2":"=====請選擇====="}'%(years,years)
           }
#     print(data_back)
#     print(data_fore)
    res_fore = requests.post(url_start, headers=headers, data=data_fore)
    res_back = requests.post(url_start, headers=headers, data=data_back)
    list_fore = json.loads(res_fore.text)
    list_back = json.loads(res_back.text)
#     pp.pprint(list_fore)
#     print(list_fore['receiveData']['totalNum'])
#     print(list_back['receiveData']['totalNum'])
    
    # crawler fore year data
    for i in list_fore['receiveData']['resultArr']:
#         print(i)
        no_fore = i['PUB_NO']
        file_fore = i['FILENAME']
        data_sfore = {'state': 'getContentByFileName',
                    'filename': file_fore
                     }
        
        # crawler content
        res_sec_fore = requests.post(url_each, headers=headers, data=data_sfore)
        raw_content_fore = json.loads(res_sec_fore.text)
        content_fore = raw_content_fore['content']['content']
        

        #crawler image
        case_no = raw_content_fore['content']['main'][77:118]
        url_img = 'https://twtmsearch.tipo.gov.tw/SS0/SS0201_q.jsp?q=1&state=doQuery&showType=2&caseType=1&caseNo=%s&caseNo2=&isReadBulletinen_US=&isReadBulletinzh_TW=true&l6=zh_TW' %(case_no)
        res_img = requests.get(url_img, headers=headers)
        raw_img = json.loads(res_img.text)['datas']['SCN2']['fullFileName']
        url_images = 'https://twtmsearch.tipo.gov.tw/imageLoad.jsp?path=%s&formatName=jpeg&pathCodeId=282_pic'%(raw_img)

        d1 = {'content': content_fore,'url_images': url_images}
        d1 = json.dumps(d1,ensure_ascii= False)

        #save text and image
        try:
            with open('%s/%s.json' %(path_resource,no_fore), 'w', encoding='utf-8') as f:
                f.write(d1)
        except FileNotFoundError as e:
            no_fore = no_fore.replace('/', '-')
            with open('%s/%s.json' %(path_resource,no_fore), 'w', encoding='utf-8') as f:
                f.write(d1)
        except OSError:
            pass


    crawler back year data
    for j in list_back['receiveData']['resultArr']:
        file_back = j['FILENAME']
        no_back = j['PUB_NO']
        data_sback = {'state': 'getContentByFileName',
                    'filename': file_back
                     }
        res_sec_back = requests.post(url_each, headers=headers, data=data_sback)
        
        raw_content_back = json.loads(res_sec_back.text)
        content_back = raw_content_back['content']['content']
        
        #crawler image
        case_no2 = raw_content_back['content']['main'][77:118]
        url_img2 = 'https://twtmsearch.tipo.gov.tw/SS0/SS0201_q.jsp?q=1&state=doQuery&showType=2&caseType=1&caseNo=%s&caseNo2=&isReadBulletinen_US=&isReadBulletinzh_TW=true&l6=zh_TW' %(case_no2)
        res_img2 = requests.get(url_img2, headers=headers)
        raw_img2 = json.loads(res_img2.text)['datas']['SCN2']['fullFileName']
        url_images2 = 'https://twtmsearch.tipo.gov.tw/imageLoad.jsp?path=%s&formatName=jpeg&pathCodeId=282_pic'%(raw_img2)
        

        d2 = {"content": content_back,"url_images": url_images2}


        try:
            with open('%s/%s.json' %(path_resource,no_back), 'w' ,encoding='utf-8') as f:
                json.dump(d2, f, ensure_ascii=False)
        except FileNotFoundError as e:
            no_back = no_back.replace('/', '-')
            with open('%s/%s.json' %(path_resource,no_back), 'w',encoding='utf-8') as f:
                json.dump(d2, f, ensure_ascii=False)
        except OSError:
            pass
        
        time.sleep()

In [ ]:
import json
import glob

filelist = glob.glob('./logo-resource/*.json')

for filename in filelist:
    with open(filename ,encoding='utf-8') as json_file:
        data = json.load(json_file)
        print(data)